In [ ]:
!pip install super_image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
import cv2
from PIL import Image
import torch
from super_image import CarnModel, ImageLoader
import numpy as np
import torchvision.transforms as transforms
import os
from google.colab import files
import shutil
from skimage.metrics import structural_similarity as ssim

def calculate_psnr(original, processed):
    """Calculate PSNR between original and processed frames"""
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return 100
    max_pixel = 255.0
    psnr_value = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr_value

def calculate_ssim(original, processed):
    """Calculate SSIM between original and processed frames"""
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    processed_gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    ssim_value, _ = ssim(original_gray, processed_gray, full=True)
    return ssim_value

def process_video(input_video_path, output_video_path, model, face_cascade):
    # Open the input video
    video = cv2.VideoCapture(input_video_path)
    if not video.isOpened():
        print(f"Error: Could not open video file {input_video_path}")
        return False

    # Get the frame properties
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))

    # Prepare the output video writer
    output_video = cv2.VideoWriter(
        output_video_path,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )

    # Define tensor-to-PIL conversion utility
    to_pil_image = transforms.ToPILImage()

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    processed_frames = 0

    # Metrics tracking
    total_psnr = 0
    total_ssim = 0
    frames_with_faces = 0

    # Process frames
    while True:
        ret, frame = video.read()
        if not ret:
            break

        original_frame = frame.copy()  # Keep original frame for comparison

        # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(
            gray_frame,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )

        if len(faces) > 0:
            frames_with_faces += 1
            # Process first detected face
            x, y, w, h = faces[0]
            face_frame = frame[y:y+h, x:x+w]

            # Convert to PIL Image
            pil_image = Image.fromarray(cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB))

            # Prepare for model
            inputs = ImageLoader.load_image(pil_image)

            # Apply super resolution
            with torch.no_grad():
                preds = model(inputs)

            # Convert back to image
            sr_image = to_pil_image(preds.squeeze(0))
            sr_image = np.array(sr_image)
            sr_image = cv2.cvtColor(sr_image, cv2.COLOR_RGB2BGR)

            # Resize to original frame size
            processed_frame = cv2.resize(sr_image, (frame_width, frame_height))

            # Calculate metrics
            psnr_value = calculate_psnr(original_frame, processed_frame)
            ssim_value = calculate_ssim(original_frame, processed_frame)

            total_psnr += psnr_value
            total_ssim += ssim_value

            output_video.write(processed_frame)
        else:
            output_video.write(frame)

        # Update progress
        processed_frames += 1
        if processed_frames % 30 == 0:  # Update every 30 frames
            progress = (processed_frames / frame_count) * 100
            print(f"\rProgress: {progress:.1f}%", end="")

    # Calculate and print average metrics
    if frames_with_faces > 0:
        avg_psnr = total_psnr / frames_with_faces
        avg_ssim = total_ssim / frames_with_faces
        print(f"\n\nQuality Metrics for {os.path.basename(input_video_path)}:")
        print(f"Average PSNR: {avg_psnr:.2f} dB")
        print(f"Average SSIM: {avg_ssim:.4f}")
        print(f"Frames with faces processed: {frames_with_faces}")
    else:
        print("\n\nNo faces were detected in the video.")

    # Clean up
    video.release()
    output_video.release()
    return True

def main():
    # Create temporary output folder
    output_folder = "/content/enhanced_videos"
    os.makedirs(output_folder, exist_ok=True)

    # Load the CARN model
    print("Loading CARN model...")
    model = CarnModel.from_pretrained('eugenesiow/carn-bam', scale=4)

    # Load face detector
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )

    # Input folder is in Colab's content directory
    input_folder = "/content/D022"

    # Get list of video files
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv')
    video_files = [f for f in os.listdir(input_folder)
                  if f.lower().endswith(video_extensions)]

    if not video_files:
        print(f"No video files found in {input_folder}")
        return

    # Process each video
    for video_file in video_files:
        input_path = os.path.join(input_folder, video_file)

        # Create output filename with '_enhanced' suffix
        filename, ext = os.path.splitext(video_file)
        output_file = f"{filename}_enhanced{ext}"
        output_path = os.path.join(output_folder, output_file)

        print(f"\nProcessing video: {video_file}")
        success = process_video(input_path, output_path, model, face_cascade)

        if success:
            print(f"Completed processing: {video_file}")
            print(f"Initiating download for: {output_file}")
            try:
                files.download(output_path)
                print(f"Download initiated for {output_file}")
            except Exception as e:
                print(f"Error initiating download: {str(e)}")
        else:
            print(f"Failed to process {video_file}")

        print("-" * 50)

    # Clean up temporary folder
    try:
        shutil.rmtree(output_folder)
    except Exception as e:
        print(f"Error cleaning up temporary folder: {str(e)}")

if __name__ == "__main__":
    main()

Loading CARN model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:680: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

pytorch_model_4x.pt:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

https://huggingface.co/eugenesiow/carn-bam/resolve/main/pytorch_model_4x.pt


...

<IPython.core.display.Javascript object>

Download initiated for sub10_c_toxic_enhanced.mp4
--------------------------------------------------

Processing video: sub14_b_toxic.mp4
Progress: 95.7%

Quality Metrics for sub14_b_toxic.mp4:
Average PSNR: 28.00 dB
Average SSIM: 0.5202
Frames with faces processed: 118
Completed processing: sub14_b_toxic.mp4
Initiating download for: sub14_b_toxic_enhanced.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated for sub14_b_toxic_enhanced.mp4
--------------------------------------------------

Processing video: sub5_d_toxic.mp4
Progress: 53.3%